In [ ]:
import os
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras as tfkeras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Input
from tensorflow.keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger
from tensorflow.keras.callbacks import ReduceLROnPlateau,LearningRateScheduler
from efficientnet.tfkeras import EfficientNetB7, EfficientNetB0, EfficientNetB4

class parameters:
    imgsize = 512
    depth = 3
    BATCH_SIZE = 4
    ILR = 1e-2
    FLR = 1e-3
    val_batch_size = 4
    initial_epoch = 10
    final_epoch = 15
    total_epochs = initial_epoch + final_epoch
    
def create_model(height, width, depth):

    # initialize the input shape
    inputShape = (height, width, depth)
    #extract features from pretrained model mobile-net
    base_model = EfficientNetB7(weights='imagenet',
                include_top=False, input_shape=inputShape)


    return base_model

def effn7(base_model, classes):

    x = base_model.output
    x = Dense(1024, activation="relu")(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    preds = Dense(classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=preds)  

    return model  

# def effnb7_finetune(base_model, classes):
#     x = base_model.output
#     # define classification layers and increase learnable parameters
#     x = Dense(1024, activation="relu")(x)
#     x = BatchNormalization()(x)
#     x = GlobalAveragePooling2D()(x)
#     x = BatchNormalization()(x)
#     preds = Dense(classes, activation='softmax')(x)
#     model = Model(inputs=base_model.input, outputs=preds)

#     return model


def train(epoch, model, classes):
  
    
    print("[LOGGING] training initial network...")
    for layer in model.layers:
        layer.trainable = False
    model = effn7(model, classes)
    # initialize the optimizer compile the model and
    opt = Adam(lr=parameters.ILR)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])    
    # print(model.summary())
    # train the network
    model_dir = '/home/output/fine_tuned_model.hdf5'
    # build the path to the training plot and training history
    plotPath = os.path.sep.join(["/home/output", "resnet_fashion_mnist.png"])
    jsonPath = os.path.sep.join(["/home/output", "resnet_fashion_mnist.json"])

    # construct the set of callbacks
    callbacks = [ModelCheckpoint(filepath=model_dir, verbose=1, save_best_only=True, monitor='val_acc'),
                 ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, verbose=1),
                 TrainingMonitor(plotPath,	jsonPath=jsonPath, startAt=0)]

    _ = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // parameters.BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // parameters.val_batch_size,
        epochs=parameters.initial_epoch,
        workers=0,
        callbacks=callbacks,
        verbose=1)
    
    print("[INFO] Fine tuning and continue train")
    print("[LOGGING] training final network...")
   
    model.trainable = True
    for layer in model.layers[:-200]:
        layer.trainable = True
    # initialize the optimizer compile the model and
    opt = Adam(lr=parameters.FLR)
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])   

    _ = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // parameters.BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // parameters.val_batch_size,
        epochs=epoch,
        workers=0,
        callbacks=callbacks,
        verbose=1)
    model.save('/home/output/effn_model_final.h5')
    return f'[info] trainig completed network for {epoch} epochs'

     

train_image_gen = ImageDataGenerator(
        rescale=1.0/255,
        zca_whitening=False,  
        rotation_range=15,  
        zoom_range = 0.1, 
        width_shift_range=0.2,  
        height_shift_range=0.2, 
        horizontal_flip=True,  
        vertical_flip=True)
 
# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_image_gen.flow_from_directory("/content/train/",
                                                      target_size=(parameters.imgsize, parameters.imgsize),
                                                      batch_size=parameters.BATCH_SIZE,
                                                      class_mode='categorical') # set as training data


validation_generator = test_datagen.flow_from_directory("/content/val/",
                                                        target_size=(parameters.imgsize, parameters.imgsize),
                                                        batch_size=parameters.val_batch_size,
                                                        class_mode='categorical'
                                                        ) # set as training data

classes = len(train_generator.class_indices)

In [ ]:
model = create_model(parameters.imgsize, parameters.imgsize, parameters.depth)
train(parameters.total_epochs, model, classes)


In [9]:
classifier = load_model('model_dogs/effn_model.h5')

In [11]:
classifier.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 64) 1728        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 64) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

block2b_drop (FixedDropout)     (None, 128, 128, 48) 0           block2b_project_bn[0][0]         
__________________________________________________________________________________________________
block2b_add (Add)               (None, 128, 128, 48) 0           block2b_drop[0][0]               
                                                                 block2a_project_bn[0][0]         
__________________________________________________________________________________________________
block2c_expand_conv (Conv2D)    (None, 128, 128, 288 13824       block2b_add[0][0]                
__________________________________________________________________________________________________
block2c_expand_bn (BatchNormali (None, 128, 128, 288 1152        block2c_expand_conv[0][0]        
__________________________________________________________________________________________________
block2c_expand_activation (Acti (None, 128, 128, 288 0           block2c_expand_bn[0][0]          
__________

__________________________________________________________________________________________________
block3a_expand_conv (Conv2D)    (None, 128, 128, 288 13824       block2g_add[0][0]                
__________________________________________________________________________________________________
block3a_expand_bn (BatchNormali (None, 128, 128, 288 1152        block3a_expand_conv[0][0]        
__________________________________________________________________________________________________
block3a_expand_activation (Acti (None, 128, 128, 288 0           block3a_expand_bn[0][0]          
__________________________________________________________________________________________________
block3a_dwconv (DepthwiseConv2D (None, 64, 64, 288)  7200        block3a_expand_activation[0][0]  
__________________________________________________________________________________________________
block3a_bn (BatchNormalization) (None, 64, 64, 288)  1152        block3a_dwconv[0][0]             
__________

block3f_bn (BatchNormalization) (None, 64, 64, 480)  1920        block3f_dwconv[0][0]             
__________________________________________________________________________________________________
block3f_activation (Activation) (None, 64, 64, 480)  0           block3f_bn[0][0]                 
__________________________________________________________________________________________________
block3f_se_squeeze (GlobalAvera (None, 480)          0           block3f_activation[0][0]         
__________________________________________________________________________________________________
block3f_se_reshape (Reshape)    (None, 1, 1, 480)    0           block3f_se_squeeze[0][0]         
__________________________________________________________________________________________________
block3f_se_reduce (Conv2D)      (None, 1, 1, 20)     9620        block3f_se_reshape[0][0]         
__________________________________________________________________________________________________
block3f_se

__________________________________________________________________________________________________
block4d_se_expand (Conv2D)      (None, 1, 1, 960)    39360       block4d_se_reduce[0][0]          
__________________________________________________________________________________________________
block4d_se_excite (Multiply)    (None, 32, 32, 960)  0           block4d_activation[0][0]         
                                                                 block4d_se_expand[0][0]          
__________________________________________________________________________________________________
block4d_project_conv (Conv2D)   (None, 32, 32, 160)  153600      block4d_se_excite[0][0]          
__________________________________________________________________________________________________
block4d_project_bn (BatchNormal (None, 32, 32, 160)  640         block4d_project_conv[0][0]       
__________________________________________________________________________________________________
block4d_dr

...

                                                                 block5c_add[0][0]                
__________________________________________________________________________________________________
block5e_expand_conv (Conv2D)    (None, 32, 32, 1344) 301056      block5d_add[0][0]                
__________________________________________________________________________________________________
block5e_expand_bn (BatchNormali (None, 32, 32, 1344) 5376        block5e_expand_conv[0][0]        
__________________________________________________________________________________________________
block5e_expand_activation (Acti (None, 32, 32, 1344) 0           block5e_expand_bn[0][0]          
__________________________________________________________________________________________________
block5e_dwconv (DepthwiseConv2D (None, 32, 32, 1344) 33600       block5e_expand_activation[0][0]  
__________________________________________________________________________________________________
block5e_bn

block5j_expand_bn (BatchNormali (None, 32, 32, 1344) 5376        block5j_expand_conv[0][0]        
__________________________________________________________________________________________________
block5j_expand_activation (Acti (None, 32, 32, 1344) 0           block5j_expand_bn[0][0]          
__________________________________________________________________________________________________
block5j_dwconv (DepthwiseConv2D (None, 32, 32, 1344) 33600       block5j_expand_activation[0][0]  
__________________________________________________________________________________________________
block5j_bn (BatchNormalization) (None, 32, 32, 1344) 5376        block5j_dwconv[0][0]             
__________________________________________________________________________________________________
block5j_activation (Activation) (None, 32, 32, 1344) 0           block5j_bn[0][0]                 
__________________________________________________________________________________________________
block5j_se

__________________________________________________________________________________________________
block6e_se_squeeze (GlobalAvera (None, 2304)         0           block6e_activation[0][0]         
__________________________________________________________________________________________________
block6e_se_reshape (Reshape)    (None, 1, 1, 2304)   0           block6e_se_squeeze[0][0]         
__________________________________________________________________________________________________
block6e_se_reduce (Conv2D)      (None, 1, 1, 96)     221280      block6e_se_reshape[0][0]         
__________________________________________________________________________________________________
block6e_se_expand (Conv2D)      (None, 1, 1, 2304)   223488      block6e_se_reduce[0][0]          
__________________________________________________________________________________________________
block6e_se_excite (Multiply)    (None, 16, 16, 2304) 0           block6e_activation[0][0]         
          

__________________________________________________________________________________________________
block6j_se_reduce (Conv2D)      (None, 1, 1, 96)     221280      block6j_se_reshape[0][0]         
__________________________________________________________________________________________________
block6j_se_expand (Conv2D)      (None, 1, 1, 2304)   223488      block6j_se_reduce[0][0]          
__________________________________________________________________________________________________
block6j_se_excite (Multiply)    (None, 16, 16, 2304) 0           block6j_activation[0][0]         
                                                                 block6j_se_expand[0][0]          
__________________________________________________________________________________________________
block6j_project_conv (Conv2D)   (None, 16, 16, 384)  884736      block6j_se_excite[0][0]          
__________________________________________________________________________________________________
block6j_pr

__________________________________________________________________________________________________
block7b_project_bn (BatchNormal (None, 16, 16, 640)  2560        block7b_project_conv[0][0]       
__________________________________________________________________________________________________
block7b_drop (FixedDropout)     (None, 16, 16, 640)  0           block7b_project_bn[0][0]         
__________________________________________________________________________________________________
block7b_add (Add)               (None, 16, 16, 640)  0           block7b_drop[0][0]               
                                                                 block7a_project_bn[0][0]         
__________________________________________________________________________________________________
block7c_expand_conv (Conv2D)    (None, 16, 16, 3840) 2457600     block7b_add[0][0]                
__________________________________________________________________________________________________
block7c_ex

In [ ]:
# import cv2
# import numpy as np
# image = '/content/test/n02086079-Pekinese/n02086079_11322.jpg'
# image = cv2.imread(image)
# image = cv2.resize(image, (512, 512))
# image = image.reshape(-1, 512, 512, 3)
# image = image / 255
# model = load_model('/content/output/effn_model_final.h5')
# predict = model.predict(image)

# np.argmax(predict)